In [5]:
import pandas as pd

In [6]:
# Load the dataset from the CSV file
df = pd.read_csv('Activities_Dataset.xls')

# Display the first few rows of the dataset
print(df.head())


           Activity Location
0     Rock Climbing    Delhi
1         Paintball    Delhi
2       Lawn Tennis    Delhi
3           Cycling    Delhi
4  Adventure Island    Delhi


In [7]:
# Check the structure and summary of the dataset
print(df.info())

# Check unique locations to understand available cities
print(df['Location'].unique())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Activity  63 non-null     object
 1   Location  63 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB
None
['Delhi' 'Chennai' 'Kolkata' 'Bengaluru' 'Mumbai' 'Hyderabad']


In [8]:
# Function to recommend activities based on location
def recommend_activities(location):
    # Filter activities for the given location
    recommendations = df[df['Location'] == location]['Activity'].tolist()
    
    # Return the list of recommended activities
    return recommendations


In [9]:
# Test with a specific destination city
destination_city = 'Location'
recommended_activities = recommend_activities(destination_city)

print(f"Recommended Activities in {destination_city}: {recommended_activities}")


Recommended Activities in Location: []


In [10]:
# Input from the user for destination city
user_city = input("Enter your destination city: ")

# Get recommendations for the user's city
recommended_activities = recommend_activities(user_city)

if recommended_activities:
    print(f"Recommended Activities in {user_city}: {recommended_activities}")
else:
    print(f"Sorry, no activities found for {user_city}.")


Sorry, no activities found for .


In [11]:
# Check if the entered city exists in the dataset
def recommend_activities_safe(location):
    # Check if location exists in the dataset
    if location in df['Location'].unique():
        recommendations = df[df['Location'] == location]['Activity'].tolist()
        return recommendations
    else:
        return []  # Return empty list if the location is not found

# Example with safe checks
user_city = input("Enter your destination city: ")

recommended_activities = recommend_activities_safe(user_city)

if recommended_activities:
    print(f"Recommended Activities in {user_city}: {recommended_activities}")
else:
    print(f"Sorry, no activities found for {user_city}.")


Sorry, no activities found for .


In [12]:
import pandas as pd

# Load the dataset from the CSV file
df = pd.read_csv('Activities_Dataset.csv')

# Display the first few rows of the dataset to check
print(df.head())

# Function to recommend activities based on location
def recommend_activities(location):
    # Filter activities for the given location
    recommendations = df[df['Location'] == location]['Activity'].tolist()
    return recommendations

# Check if the entered city exists in the dataset (safe function)
def recommend_activities_safe(location):
    if location in df['Location'].unique():
        recommendations = df[df['Location'] == location]['Activity'].tolist()
        return recommendations
    else:
        return []

# Interactive input from the user for destination city
user_city = input("Enter your destination city: ")

# Get recommendations for the user's city
recommended_activities = recommend_activities_safe(user_city)

if recommended_activities:
    print(f"Recommended Activities in {user_city}: {recommended_activities}")
else:
    print(f"Sorry, no activities found for {user_city}.")


FileNotFoundError: [Errno 2] No such file or directory: 'Activities_Dataset.csv'

In [ ]:
#BRUHHH

In [51]:
import pandas as pd
import numpy as np

# Assuming you have a CSV file that represents user-activity interactions
# with columns: user_id, activity_id, interaction (1 for interacted, 0 for not interacted)
interaction_data = pd.read_csv('Activities_Dataset.csv')

print(interaction_data.head())


           Activity Location
0     Rock Climbing    Delhi
1         Paintball    Delhi
2       Lawn Tennis    Delhi
3           Cycling    Delhi
4  Adventure Island    Delhi


In [59]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense

# Load interaction data
interaction_data = pd.read_csv('Activities_Dataset.csv')

# Display the columns and first few rows
print(interaction_data.columns.tolist())
print(interaction_data.head())

# Encode Activity and Location
activity_encoder = LabelEncoder()
interaction_data['activity_encoded'] = activity_encoder.fit_transform(interaction_data['Activity'])

location_encoder = LabelEncoder()
interaction_data['location_encoded'] = location_encoder.fit_transform(interaction_data['Location'])

# Prepare feature matrix for content-based filtering
X = interaction_data[['activity_encoded', 'location_encoded']]

# Create a dummy interaction column (e.g., all ones) for training purposes
interaction_data['interaction'] = 1  # Assuming every entry is an interaction

# Prepare training labels
y = interaction_data['interaction']

# Split data for training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define embedding dimensions
n_activities = interaction_data['activity_encoded'].nunique()
n_locations = interaction_data['location_encoded'].nunique()
embedding_dim = 50

# Build the content-based filtering model
activity_input = Input(shape=(1,), name='activity_input')
activity_embedding = Embedding(input_dim=n_activities, output_dim=embedding_dim, name='activity_embedding')(activity_input)
activity_vector = Flatten(name='flatten_activity')(activity_embedding)

location_input = Input(shape=(1,), name='location_input')
location_embedding = Embedding(input_dim=n_locations, output_dim=embedding_dim, name='location_embedding')(location_input)
location_vector = Flatten(name='flatten_location')(location_embedding)

# Concatenate the activity and location vectors
concatenated = Concatenate()([activity_vector, location_vector])
dense_1 = Dense(128, activation='relu')(concatenated)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(1, activation='sigmoid')(dense_2)

model = Model(inputs=[activity_input, location_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([X_train['activity_encoded'], X_train['location_encoded']], y_train, epochs=10, batch_size=64, validation_data=([X_test['activity_encoded'], X_test['location_encoded']], y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate([X_test['activity_encoded'], X_test['location_encoded']], y_test)
print(f"Test Accuracy: {test_accuracy}")

# Recommend activities based on input
def recommend_activities(activity_name, location_name, top_n=5):
    activity_encoded = activity_encoder.transform([activity_name])
    location_encoded = location_encoder.transform([location_name])
    
    predictions = model.predict([np.repeat(activity_encoded, n_locations), np.arange(n_locations)])
    top_indices = np.argsort(predictions[:, 0])[-top_n:]
    
    recommended_activities = activity_encoder.inverse_transform(top_indices)
    return recommended_activities

# Get recommendations for a specific activity and location
recommendations = recommend_activities('Rock Climbing', 'Delhi', top_n=5)
print(f"Recommended activities for Rock Climbing in Delhi: {recommendations}")


['Activity', 'Location']
           Activity Location
0     Rock Climbing    Delhi
1         Paintball    Delhi
2       Lawn Tennis    Delhi
3           Cycling    Delhi
4  Adventure Island    Delhi
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3400 - loss: 0.6960 - val_accuracy: 0.7692 - val_loss: 0.6884
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.8000 - loss: 0.6856 - val_accuracy: 0.9231 - val_loss: 0.6797
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 1.0000 - loss: 0.6758 - val_accuracy: 1.0000 - val_loss: 0.6713
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 1.0000 - loss: 0.6661 - val_accuracy: 1.0000 - val_loss: 0.6625
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 1.0000 - loss: 0.6563 - val_accuracy: 1.0000 - val_loss: 0.6533
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 1.0000 - loss: 0.6460 - val_accuracy: 1.0000 - val_loss: 0.6436
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - ac